In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
import shutil

In [4]:
def rep(s):
    x = s.replace("_ClinicalData_","_MR_")
    return x
sorted_labels['New Label'] = sorted_labels['Label'].apply(rep) + '.nii'

In [6]:
#read oasis_data
labels = pd.read_csv("oasis_label_2.csv")
# days since MRI
labels['Days since MRI'] = pd.to_numeric(labels['Label'].str[-4:])
# month and two month since MRI
labels['Months since MRI'] = np.floor(labels['Days since MRI']/30)
labels['2 months since MRI'] = np.floor(labels['Days since MRI']/60)
# drop subjects who have two mri at the same day 
new = labels
new = new.drop_duplicates(subset=['Subject','2 months since MRI'])
# sort by subject and then months since mri
sorted_labels = new.sort_values(by=['Subject','Months since MRI'])

# need to change format since MRI format is OAS30073_MR_d3670.nii, 
# but nameID on excel file format is OAS30073_ClinicalData_d3670
# OAS30001_ClinicalData_d0000 -> OAS30001_MR_d0000.nii

sorted_labels['New Label'] = sorted_labels['Label'].apply(rep) + '.nii'



# divide sorted_label into 3 label-normal, uncertain and AD_labels
normal_labels = sorted_labels[sorted_labels['dx1'] == 'Cognitively normal']
uncertain_labels = sorted_labels[sorted_labels['dx1'] == 'uncertain dementia']
AD_labels = sorted_labels[sorted_labels['dx1'] == 'AD Dementia']

######################################################################################################################

# oasis_image_data_dates.csv matches sorted_labels
image_data = pd.read_csv("oasis_image_data_dates.csv")


def match(df, col1 = 'subject',col2 = 'days'):
    df1=sorted_labels[sorted_labels["Subject"]==df[col1]]
    loc = (np.abs(df1["Days since MRI"] - df[col2])).argmin()            
    low_limit = df[col2] - 180
    high_limit = df[col2] + 180
    if sorted_labels.loc[loc]['Days since MRI'] < high_limit and sorted_labels.loc[loc]['Days since MRI'] > low_limit:
#         #here return dx1 other than the nearest day
#         return int(sorted_labels.loc[loc]['Days since MRI'])
        return sorted_labels.loc[loc]['dx1']

    else:
        return None
image_data["match_label"] = image_data.apply(match,col1 = 'subject',col2 = 'days',axis = 1)
image_data.head()


# def match(day_file,subj_file,excel):
#     df1=excel[excel["Subject"]==subj_file]
#     loc = (np.abs(df1["Days since MRI"] - day_file)).argmin()            
#     low_limit = day_file - 180
#     high_limit = day_file + 180
#     if excel.loc[loc]['Days since MRI'] < high_limit and excel.loc[loc]['Days since MRI'] > low_limit:
#         return excel.loc[loc]['Days since MRI']
#     else:
#         return None
# df['nearest'] = df.apply(lambda x: match(x['days'],x['subject'],sorted_labels),axis=1) 
# df.head()

,Count,actual file name,subject,type,file_name_day,day label,days,match_label
0,17814,OAS30001_MR_d0129.nii.gz,OAS30001,MR,d0129.nii.gz,d0129,129,Cognitively normal
1,15818,OAS30001_MR_d0757.nii.gz,OAS30001,MR,d0757.nii.gz,d0757,757,Cognitively normal
2,16079,OAS30001_MR_d2430.nii.gz,OAS30001,MR,d2430.nii.gz,d2430,2430,None
3,17075,OAS30001_MR_d3132.nii.gz,OAS30001,MR,d3132.nii.gz,d3132,3132,Cognitively normal
4,22537,OAS30002_MR_d0371.nii.gz,OAS30002,MR,d0371.nii.gz,d0371,371,None


In [7]:
image_data.groupby('match_label').size()

match_label
0.5 in memory only                          7
AD Dementia                               193
AD dem Language dysf after                  5
AD dem Language dysf prior                  2
AD dem Language dysf with                   3
AD dem cannot be primary                    1
AD dem distrubed social- after              2
AD dem distrubed social- with               2
AD dem visuospatial- with                   1
AD dem w/CVD contribut                      1
AD dem w/CVD not contrib                    3
AD dem w/PDI after AD dem contribut         1
AD dem w/PDI after AD dem not contrib       4
AD dem w/depresss, not contribut            1
AD dem w/depresss- contribut                6
AD dem w/depresss- not contribut           10
AD dem w/oth (list B) contribut             2
AD dem w/oth (list B) not contrib           2
Cognitively normal                       1281
DAT                                         2
DLBD, primary                               1
DLBD- primary         

In [9]:
image_data.head()

,Count,actual file name,subject,type,file_name_day,day label,days,match_label
0,17814,OAS30001_MR_d0129.nii.gz,OAS30001,MR,d0129.nii.gz,d0129,129,Cognitively normal
1,15818,OAS30001_MR_d0757.nii.gz,OAS30001,MR,d0757.nii.gz,d0757,757,Cognitively normal
2,16079,OAS30001_MR_d2430.nii.gz,OAS30001,MR,d2430.nii.gz,d2430,2430,None
3,17075,OAS30001_MR_d3132.nii.gz,OAS30001,MR,d3132.nii.gz,d3132,3132,Cognitively normal
4,22537,OAS30002_MR_d0371.nii.gz,OAS30002,MR,d0371.nii.gz,d0371,371,None


In [19]:
normal_data = image_data[image_data['match_label'] == 'Cognitively normal']
AD_data = image_data[image_data['match_label'] == 'AD Dementia']
uncertain_data = image_data[image_data['match_label'] == 'uncertain dementia']

normal_data.count()

Count               1281
actual file name    1281
subject             1281
type                1281
file_name_day       1281
day label           1281
days                1281
match_label         1281
dtype: int64

In [32]:
src = '/Users/Nand/Desktop/Deep_Learning/Project/fs_t1/'

for index, row in uncertain_data.iterrows():  
    srcpath = os.path.join(src, row['actual file name'])
    dstpath = '/Users/Nand/Desktop/Deep_Learning/Project/Uncertain_Data/'

    shutil.copy2(srcpath, dstpath)    
    

In [29]:
src = '/Users/Nand/Desktop/Deep_Learning/Project/fs_t1/'

for index, row in AD_data.iterrows():  
    srcpath = os.path.join(src, row['actual file name'])
    dstpath = '/Users/Nand/Desktop/Deep_Learning/Project/AD_Data/'

    shutil.copy2(srcpath, dstpath)  

(29, Count                                  19156
actual file name    OAS30007_MR_d2722.nii.gz
subject                             OAS30007
type                                      MR
file_name_day                   d2722.nii.gz
day label                              d2722
days                                    2722
match_label               uncertain dementia
Name: 29, dtype: object)
(41, Count                                  17441
actual file name    OAS30013_MR_d0102.nii.gz
subject                             OAS30013
type                                      MR
file_name_day                   d0102.nii.gz
day label                              d0102
days                                     102
match_label               uncertain dementia
Name: 41, dtype: object)
(48, Count                                  17885
actual file name    OAS30016_MR_d0021.nii.gz
subject                             OAS30016
type                                      MR
file_name_day                   d00

In [ ]:
src = '/Users/Nand/Desktop/Deep_Learning/Project/fs_t1/'

for index, row in normal_data.iterrows():  
    srcpath = os.path.join(src, row['actual file name'])
    dstpath = '/Users/Nand/Desktop/Deep_Learning/Project/normal_data/'

    shutil.copy2(srcpath, dstpath)  